In [2]:
################################################
# Ayoub Sikouky
# Title: SISR Using a Multi Model System
# Edit Date: 22/04/2022
################################################
import os
from pathlib import Path
import tensorflow as tf
from typing import List

## I - Data

### 1 : Helper Functions

In [9]:
########################### Data ###########################
def download_data(f_name, dst, source_url):
    """
    Download and unzip datastes.
    #### Parameters:
    f_name: zip file name.
    dst: The destination directory where the files will be stored.
    source_url: The URL for the dataset.
    """
    file_url = os.path.join(source_url, f_name)
    dst_abspath = os.path.abspath(dst)
    tf.keras.utils.get_file(f_name, file_url, cache_subdir=dst_abspath, extract=True)
    # Removing the zip file.
    os.remove(os.path.join(dst_abspath, f_name)) 


def get_paths(dirs: List[str], path_type):
    """
    Retuns a list of images' paths inside a directories 
    that have "path_type" in their name.
    #### Parameters:
    dirs: The list of directories to be searched.
    path_type: The "characters" to selcet which directories to search for images.
    """
    if not isinstance(dirs, list):
      raise Exception("'dirs' argument must be a list.")

    paths_list = [] # List of paths in "dirs"
    subpaths_lists = [] # List of lists of images' paths.
    for d in dirs:
        if path_type in str(d):
            paths_list.append(d)
    paths_list.sort()
    
    if len(paths_list) == 1:
        img_paths = [str(f) for f in paths_list[0].glob("**/*.png")]
        img_paths.sort()
        return img_paths
    else:
      for main_path in paths_list:
        tmp_subpath_list = [str(f) for f in main_path.glob("**/*.png") if f.is_file]
        tmp_subpath_list.sort()
        subpaths_lists.append(tmp_subpath_list)

    return subpaths_lists

### 2: Preparing the Data

In [10]:
# Downloading the datasets used in training and validation.
if not os.path.exists("Original_Datasets"):

    DIV2K_URL = "http://data.vision.ee.ethz.ch/cvl/DIV2K/"
    DIV2K_FILE_NAMES = ["DIV2K_train_HR.zip", "DIV2K_valid_HR.zip", 
                        "DIV2K_train_LR_bicubic_X2.zip", "DIV2K_valid_LR_bicubic_X2.zip"]
    dst_dir = "Original_Datasets/DIV2K"

    for file in DIV2K_FILE_NAMES:
        download_data(file, dst_dir, DIV2K_URL)

#### 2.1 Data Paths

In [4]:
# Check if colab is used.
using_google_colab = "google.colab" in str(get_ipython())

## When Using Google Colab
if using_google_colab:
  from google.colab import drive
  drive.mount('/content/drive')
  colab_path = "/content/drive/MyDrive/ARU's Master/Application Of Machine Learning/Assignments/Super_Resolution/"
  ALL_DATA_PATH = Path(colab_path+"Original_Datasets")
  MODELS_PATH = colab_path + "Models/"

else:
## When Using Local Machine.
  ALL_DATA_PATH = Path(r"Original_Datasets")
  MODELS_PATH = "Models/"

In [10]:
BATCH_SIZE = 16
SCALE = 2 # The scale of the downsample images used.

# Getting all sub directories inside the working directory.
all_paths = ALL_DATA_PATH.glob("**/*")
dirs = [d for d in all_paths if d.is_dir()]

# Selecting HR (High Resolution) directories.
all_sub_paths = get_paths(dirs, "HR")

# 
HR_TRAIN_PATH, HR_VALID_PATH = all_sub_paths[0], all_sub_paths[1]
print(f"HR Train Len: {len(HR_TRAIN_PATH)}")
print(f"HR Valid Len: {len(HR_VALID_PATH)}")
print(f"HR Train Paths Example:\n{HR_TRAIN_PATH[:2]}")

LR_TRAIN_PATH, LR_VALID_PATH = get_paths(dirs, f"X{SCALE}")
print(f"LR Train Len: {len(LR_TRAIN_PATH)}")
print(f"LR Valid Len: {len(LR_VALID_PATH)}")
print(f"LR Train Paths Example:\n{LR_TRAIN_PATH[:2]}")

HR Train Len: 800
HR Valid Len: 100
HR Train Paths Example:
['Original_Datasets/DIV2K/DIV2K_train_HR/0001.png', 'Original_Datasets/DIV2K/DIV2K_train_HR/0002.png']
LR Train Len: 800
LR Valid Len: 100
LR Train Paths Example:
['Original_Datasets/DIV2K/DIV2K_train_LR_bicubic/X2/0001x2.png', 'Original_Datasets/DIV2K/DIV2K_train_LR_bicubic/X2/0002x2.png']


#### 2.2 Creating the datasets used with the model